In [1]:
import fasttext
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import fasttext.util
import os
import scipy.signal

from sklearn.metrics import classification_report
from tqdm import tqdm
from vmdpy import VMD
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

os.chdir('../')
os.chdir('data/')

## Functions needed for execution

format_text() takes the dataframe and the column index which contains the text that needs to be cleaned, in this case the cleaning process was focussed for Tweets which include removal of special charactors, links and numbers.

In [2]:
def format_text(df,col):
      #Remove @ tags
      comp_df = df.copy()

      # remove all the punctuation
      comp_df[col] = comp_df[col].str.replace(r'(@\w*)','')

      #Remove URL
      comp_df[col] = comp_df[col].str.replace(r"http\S+", "")

      #Remove # tag and the following words
      comp_df[col] = comp_df[col].str.replace(r'#\w+',"")

      #Remove all non-character
      comp_df[col] = comp_df[col].str.replace(r"[^a-zA-Z ]","")

      # Remove extra space
      comp_df[col] = comp_df[col].str.replace(r'( +)'," ")
      comp_df[col] = comp_df[col].str.strip()

      # Change to lowercase
      comp_df[col] = comp_df[col].str.lower()
      comp_df[col] = comp_df[col].str.replace('httpurl', '')
      return comp_df

From the given modes, using the SciPy package the mode containing the maximum energy will be selected.

In [3]:
def energy(u):
# Estimate PSD `S_xx_welch` at discrete frequencies `f_welch`
    f_welch, S_xx_welch = scipy.signal.welch(u)
    # Integrate PSD over spectral bandwidth
    # to obtain signal power `P_welch`
    df_welch = f_welch[1] - f_welch[0]
    return np.sum(S_xx_welch) * df_welch

In [4]:
def maxvdm(f):
    alpha = 2     
    tau = 0.            
    K = 2       
    DC = 0             
    init = 1           
    tol = 1e-8
    u, u_hat, omega = VMD(f, alpha, tau, K, DC, init, tol) 
    energy_array=[]
    for i in u:
        energy_array.append(energy(i))
    ind = np.argmax(energy_array)
    return u[ind]

## Training Data:

Loading and Pre-Processing the Tweets Data

In [5]:
train = pd.read_csv('train.tsv',sep='\t')

In [6]:
train = format_text(train,'Text')
X = train['Text'].tolist()
Y_train = train['Label']

Converting String Labels into Numeric Values with LabelEncoder

In [7]:
le = LabelEncoder()
le.fit(Y_train)
Y_train = le.transform(Y_train)

Computing the TF-IDF vectors from the given corpus of training data

In [8]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(X).toarray()

From the functions section, the maxvdm() which extracts K modes and returns the mode with the highest energy is called on every line.

In [9]:
X_data = [maxvdm(i) for i in tqdm(features)]

100%|██████████| 6936/6936 [14:15<00:00,  8.10it/s]


In [10]:
df = pd.DataFrame(X_data)
df['l'] = Y_train

## Testing Data:

The process of testing is similar to the above-mentioned training method, the same object tfidf and labelencoder must be called.

In [11]:
test = pd.read_csv('test.tsv',sep='\t',header=None)
test = format_text(test,1)
X_test = test[1].tolist()
Y_test = le.transform(test[2])

In [12]:
features_test = tfidf.transform(X_test).toarray()
X_test = [maxvdm(i) for i in tqdm(features_test)]

100%|██████████| 2000/2000 [04:00<00:00,  8.31it/s]


## Evaluating Model Performance:

In [13]:
lr = LogisticRegression(random_state=0)
lr.fit(features,Y_train)

LogisticRegression(random_state=0)

In [16]:
y_pred = lr.predict(X_test)

In [17]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.28      0.42       944
           1       0.60      0.97      0.74      1056

    accuracy                           0.64      2000
   macro avg       0.74      0.62      0.58      2000
weighted avg       0.73      0.64      0.59      2000

